# Getting started with astir

## 0. Load necessary libraries

In [23]:
# !pip install -e ../../..
from astir.data_readers import from_csv_yaml
import pandas as pd
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## 1. Load data

We start by reading expression data in the form of a csv file and marker gene information in the form of a yaml file:

In [24]:
expression_mat_path = "../../../astir/tests/test-data/sce.csv"
yaml_marker_path = "../../../astir/tests/test-data/jackson-2020-markers.yml"

.. note:: 
    Expression data should already be cleaned and normalized. In our workflow, we perform this by a log-transformation of the data with a pseudocount of 1 (i.e. `log(x+1)`), followed by winsorization at the (1%,99%) percentiles.

We can view both the expression data and marker data:

In [25]:
!head -n 20 ../../../astir/tests/test-data/jackson-2020-markers.yml


cell_states:
  RTK_signalling:
    - Her2
    - EGFR
  proliferation:
    - Ki-67
    - phospho Histone
  mTOR_signalling:
    - phospho mTOR
    - phospho S6
  apoptosis:
    - cleaved PARP
    - Cleaved Caspase3

cell_types:
  Stromal:
    - Vimentin
    - Fibronectin
  B cells:


In [26]:
pd.read_csv(expression_mat_path, index_col=0)[['EGFR','E-Cadherin', 'CD45', 'Cytokeratin 5']].head()

,EGFR,E-Cadherin,CD45,Cytokeratin 5
BaselTMA_SP41_186_X5Y4_3679,0.346787,0.938354,0.227730,0.095283
BaselTMA_SP41_153_X7Y5_246,0.833752,1.364884,0.068526,0.124031
BaselTMA_SP41_20_X12Y5_197,0.110006,0.177361,0.301222,0.052750
BaselTMA_SP41_14_X1Y8_84,0.282666,1.122174,0.606941,0.093352
BaselTMA_SP41_166_X15Y4_266,0.209066,0.402554,0.588273,0.064545


Then we can create an astir object using the `from_csv_yaml` function. For more data loading options, see the data loading tutorial.

In [27]:
ast = from_csv_yaml(expression_mat_path, marker_yaml=yaml_marker_path)
print(ast)

Astir object with 6 cell types, 4 cell states, and 100 cells.


## 2. Fitting cell types

To fit cell types, simply call

In [28]:
ast.fit_type()

training astir:  50%|█████     | 5/10 [00:00<00:00, 48.78epochs/s]

---------- Astir Training 1/5 ----------


training astir:   0%|          | 0/10 [00:00<?, ?epochs/s]

Done!
---------- Astir Training 2/5 ----------
Done!
---------- Astir Training 3/5 ----------


training astir:   0%|          | 0/10 [00:00<?, ?epochs/s]

Done!
---------- Astir Training 4/5 ----------
Done!
---------- Astir Training 5/5 ----------


training astir: 100%|██████████| 10/10 [00:00<00:00, 53.92epochs/s]

Done!



/Users/kierancampbell/toronto/astir/astir/astir.py:164: UserWarning: Maximum epochs reached. More iteration may be needed to complete the training.
  warnings.warn(msg)


.. note:: 
    **Controlling inference**
    There are many different options for controlling inference in the `fit_type` function, including
    `max_epochs` (maximum number of epochs to train),
    `learning_rate` (ADAM optimizer learning rate),
    `batch_size` (minibatch size),
    `delta_loss` (stops iteration once the change in loss falls below this value),
    `n_inits` (number of restarts using random initializations).
    For full details, see the function documentation.

We can then get cell type assignment probabilities by calling

In [29]:
assignments = ast.get_celltype_probabilities()
assignments

,Stromal,B cells,T cells,Macrophage,Epithelial (basal),Epithelial (luminal),Other
BaselTMA_SP41_186_X5Y4_3679,0.221248,0.098653,0.229925,0.412613,0.003278,0.001213,0.033069
BaselTMA_SP41_153_X7Y5_246,0.000633,0.002620,0.002430,0.001141,0.085851,0.904860,0.002465
BaselTMA_SP41_20_X12Y5_197,0.342378,0.145090,0.171638,0.194915,0.003327,0.001646,0.141006
BaselTMA_SP41_14_X1Y8_84,0.017431,0.047667,0.143390,0.027049,0.097624,0.642212,0.024626
BaselTMA_SP41_166_X15Y4_266,0.248586,0.169628,0.212497,0.325423,0.003296,0.001028,0.039542
...,...,...,...,...,...,...,...
BaselTMA_SP41_114_X13Y4_1057,0.006040,0.027174,0.019759,0.010380,0.484074,0.433319,0.019253
BaselTMA_SP41_141_X11Y2_2596,0.008376,0.026720,0.019474,0.014057,0.276350,0.629712,0.025312
BaselTMA_SP41_100_X15Y5_170,0.165827,0.146199,0.155665,0.187260,0.059510,0.063353,0.222186
BaselTMA_SP41_14_X1Y8_2604,0.289735,0.095814,0.222850,0.355587,0.001338,0.000424,0.034252


where each row corresponds to a cell, and each column to a cell type, with the entry being the probability of that cell belonging to a particular cell type.

To fetch an array corresponding to the most likely cell type assignments, call

In [30]:
ast.get_celltypes()

,cell_type
BaselTMA_SP41_186_X5Y4_3679,Unknown
BaselTMA_SP41_153_X7Y5_246,Epithelial (luminal)
BaselTMA_SP41_20_X12Y5_197,Unknown
BaselTMA_SP41_14_X1Y8_84,Unknown
BaselTMA_SP41_166_X15Y4_266,Unknown
...,...
BaselTMA_SP41_114_X13Y4_1057,Unknown
BaselTMA_SP41_141_X11Y2_2596,Unknown
BaselTMA_SP41_100_X15Y5_170,Unknown
BaselTMA_SP41_14_X1Y8_2604,Unknown


### Cell type diagnostics

It is important to run diagnostics to ensure that cell types express their markers at higher levels than other cell types. To do this, run the `diagnostics_celltype()` function, which will alert to any issues if a cell type doesn't express its marker signficantly higher than an alternative cell type (for which that protein isn't a marker):

In [31]:
ast.diagnostics_celltype().head(n=10)

,feature,should be expressed higher in,than,mean cell type 1,mean cell type 2,p-value,note
0,CD20,B cells,Epithelial (basal),0.618366,0.049826,inf,Only 1 cell in a type: comparison not possible
1,CD20,B cells,Epithelial (luminal),0.618366,0.077568,inf,Only 1 cell in a type: comparison not possible
2,CD45,B cells,Epithelial (basal),0.437730,0.085557,inf,Only 1 cell in a type: comparison not possible
3,CD45,B cells,Epithelial (luminal),0.437730,0.068204,inf,Only 1 cell in a type: comparison not possible
4,Cytokeratin 5,Epithelial (basal),B cells,1.095466,0.166079,inf,Only 1 cell in a type: comparison not possible
5,pan Cytokeratin,Epithelial (basal),B cells,2.139420,0.038145,inf,Only 1 cell in a type: comparison not possible
6,E-Cadherin,Epithelial (basal),B cells,1.511148,0.095753,inf,Only 1 cell in a type: comparison not possible
7,Cytokeratin 14,Epithelial (basal),B cells,1.332083,0.133421,inf,Only 1 cell in a type: comparison not possible
8,Her2,Epithelial (basal),B cells,0.900730,0.144622,inf,Only 1 cell in a type: comparison not possible
9,Cytokeratin 5,Epithelial (luminal),B cells,0.083562,0.166079,inf,Only 1 cell in a type: comparison not possible


.. note:: 
    In this tutorial, we end up with many "Only 1 cell in a type: comparison not possible" notes - this is simply because the small dataset size results in only a single cell assigned to many types, making statistical testing infeasible.

Calling `ast.diagnostics_celltype()` returns a `pd.DataFrame`, where each column corresponds to a particular protein and two cell types, with a warning if the protein is not expressed at higher levels in the cell type for which it is a marker than the cell type for which it is not. `None` is returned by the function if no issues are found.

The diagnostics:

1. Iterates through every cell type and every marker for that cell type

2. Given a cell type *c* and marker *g*, find the set of cell types *D* that don't have *g* as a marker

3. For each cell type *d* in *D*, perform a t-test between the expression of marker *g* in *c* vs *d*

4. If *g* is not expressed significantly higher (at significance *alpha*), output a diagnostic explaining this for further investigation.

If multiple issues are found, the markers and cell types may need refined.

## 3. Fitting cell state

Similarly as before, to fit cell state, call

In [32]:
ast.fit_state()

---------- Astir Training 1/5 ----------


training astir:   0%|          | 0/100 [00:00<?, ?epochs/s]

---------- Astir Training 2/5 ----------


training astir:   0%|          | 0/100 [00:00<?, ?epochs/s]

---------- Astir Training 3/5 ----------


training astir:   0%|          | 0/100 [00:00<?, ?epochs/s]

---------- Astir Training 4/5 ----------


training astir:   0%|          | 0/100 [00:00<?, ?epochs/s]

---------- Astir Training 5/5 ----------


training astir: 100%|██████████| 100/100 [00:00<00:00, 956.31epochs/s]
/Users/kierancampbell/toronto/astir/astir/models/cellstate.py:134: UserWarning: Delta loss batch size is greater than the number of epochs
  warnings.warn("Delta loss batch size is greater than the number of epochs")
training astir: 0epochs [00:00, ?epochs/s]
/Users/kierancampbell/toronto/astir/astir/astir.py:250: UserWarning: Maximum epochs reached. More iteration may be needed to complete the training.
  warnings.warn(msg)


and cell state assignments can be inferred via

In [33]:
states = ast.get_cellstates()
states

,RTK_signalling,proliferation,mTOR_signalling,apoptosis
BaselTMA_SP41_186_X5Y4_3679,-0.055859,-0.248295,0.081537,-0.440844
BaselTMA_SP41_153_X7Y5_246,0.026950,0.334187,-0.029829,-0.233749
BaselTMA_SP41_20_X12Y5_197,-0.107626,0.441981,-0.379208,-0.184434
BaselTMA_SP41_14_X1Y8_84,-0.171197,-0.700986,0.160324,-0.510929
BaselTMA_SP41_166_X15Y4_266,0.399416,-0.046144,-0.352460,-0.801181
...,...,...,...,...
BaselTMA_SP41_114_X13Y4_1057,0.602089,-0.176759,0.170743,0.660372
BaselTMA_SP41_141_X11Y2_2596,-0.267246,1.100570,-0.607341,-0.432088
BaselTMA_SP41_100_X15Y5_170,-0.506588,-0.183379,0.653195,-1.094012
BaselTMA_SP41_14_X1Y8_2604,-1.027609,0.369216,0.472711,-0.676119


## 4. Saving results

Both cell type and cell state information can easily be saved to disk via

In [34]:
ast.type_to_csv("cell-types.csv")
ast.state_to_csv("cell-states.csv")

In [35]:
!head -n 3 cell-types.csv

,Stromal,B cells,T cells,Macrophage,Epithelial (basal),Epithelial (luminal),Other
BaselTMA_SP41_186_X5Y4_3679,0.22124830089944308,0.09865300135667737,0.22992503215403207,0.4126134882648775,0.0032782063344407465,0.0012130607139379573,0.03306891027659129
BaselTMA_SP41_153_X7Y5_246,0.0006330683152635336,0.002619607819580104,0.002430415112729734,0.0011409748360524533,0.08585110171656299,0.9048598658487834,0.002464966351027731


In [36]:
!head -n 3 cell-states.csv

,RTK_signalling,proliferation,mTOR_signalling,apoptosis
BaselTMA_SP41_186_X5Y4_3679,-0.055859283,-0.24829505,0.08153685,-0.44084388
BaselTMA_SP41_153_X7Y5_246,0.026950097,0.33418685,-0.029828804,-0.23374897


where the first (unnamed) column always corresponds to the cell name/ID.

## 5. Accessing internal functions and data

Data stored in `astir` objects is in the form of an `SCDataSet`. These can be retrieved via

In [37]:
celltype_data = ast.get_type_dataset()
celltype_data

and similarly for cell state via `ast.get_state_dataset()`.

These have several helper functions to retrieve relevant information to the dataset:

In [38]:
celltype_data.get_cells()[0:4] # cell names

['BaselTMA_SP41_186_X5Y4_3679',
 'BaselTMA_SP41_153_X7Y5_246',
 'BaselTMA_SP41_20_X12Y5_197',
 'BaselTMA_SP41_14_X1Y8_84']

In [39]:
celltype_data.get_classes() # cell type names

['Stromal',
 'B cells',
 'T cells',
 'Macrophage',
 'Epithelial (basal)',
 'Epithelial (luminal)']

In [40]:
print(celltype_data.get_n_classes()) # number of cell types
print(celltype_data.get_n_features()) # number of features / proteins

6
14


In [41]:
celltype_data.get_exprs() # Return a torch tensor corresponding to the expression data used

tensor([[0.0953, 0.7714, 0.4610,  ..., 0.2277, 0.0571, 2.2151],
        [0.1240, 3.9632, 0.3828,  ..., 0.0685, 0.4853, 0.5026],
        [0.0527, 0.0481, 0.0203,  ..., 0.3012, 0.0359, 0.8102],
        ...,
        [0.0965, 1.4923, 0.6964,  ..., 0.0869, 0.0000, 0.7593],
        [0.0557, 0.4174, 0.1284,  ..., 0.2395, 0.0823, 2.2464],
        [0.0803, 0.2552, 0.2085,  ..., 0.2476, 0.0000, 3.1238]],
       dtype=torch.float64)